In [ ]:
from train import *
from params import TrainingParams
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

In [ ]:
params = TrainingParams()
params.train_path='/home/ubuntu/work/data/unlabeled2017/*.jpg'
params.style_path='data/starry_night.jpg'
params.batch_size = 32
params.num_epoch = 2
params.learn_rate = 0.001
params.total_train_sample = 1024

In [ ]:
def report_loss(batch, epoch, total_cost):
    cost_history.append(total_cost)
    plt.plot(cost_history)
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
cost_history = []
train(params, report_fn=report_loss)